In [5]:
import os
import re
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph

# Texto de entrada
texto = """
A análise do levantamento de preços das mensalidades nas principais escolas particulares de
Belo Horizonte revela uma série de nuances relevantes no contexto educacional e econômico da
região. A variação exorbitante de 156% entre as escolas proporciona um panorama do acesso à
educação de qualidade e suas implicações socioeconômicas. ### 1. **Estratificação do Mercado
Educacional** A disparidade de preços reflete, em certa medida, a estratificação do mercado
educacional. Escolas com mensalidades mais elevadas, como o Colégio Bernoulli e o Colégio
Magnum, frequentemente oferecem não apenas uma infraestrutura superior, mas também métodos
pedagógicos diferenciados, atividades extracurriculares qualificadas e, em muitos casos, um
ensino bilíngue. Essa diferenciação sugere que a educação se tornou um bem de consumo, onde a
qualidade percebida está diretamente ligada ao custo, perpetuando ciclos de desigualdade
social. ### 2. **Acesso à Educação** A variação significativa nos valores das mensalidades
levanta questões sobre a acessibilidade da educação de qualidade. Com preços que podem
ultrapassar os R$ 3 mil, torna-se evidente que o acesso a essas instituições é restrito a uma
parcela da população que possui meios financeiros para suportar esses custos. Para muitas
famílias, especialmente em um cenário econômico desafiador, esse investimento pode se tornar um
fardo, resultando em dilemas sobre a escolha da escola mais adequada versus a viabilidade
financeira. ### 3. **Impacto da Educação Bilingue** A exclusão de instituições bilíngues como
Maple Bear e Fundação Torino do levantamento destaca outro aspecto crucial: a crescente demanda
por educação bilíngue, que se tornou um diferencial no mercado. Essas escolas, que não foram
incluídas na pesquisa, frequentemente atendem a uma clientela que busca uma formação mais
globalizada, levando em consideração que o domínio de línguas estrangeiras é um diferencial
importante no mundo atual. Este fenômeno gera uma nova camada de diferenciação de mercado, onde
instituições que oferecem tal ensino se estabelecem em uma faixa ainda mais elevada, tanto em
termos de mensalidade quanto de prestígio. ### 4. **Consequências para a Política
Educacional** Esses dados podem servir como um alerta para formuladores de políticas públicas.
A educação, enquanto direito fundamental, deve ser acessível a todos, independentemente da
classe social. A educação de qualidade deve caminhar junto com estratégias que visem integrar
diferentes grupos sociais, promovendo não apenas a inclusão em termos de acesso, mas também em
qualidade e diversidade no ambiente escolar. ### 5. **Prospectos Futuro** A alta variação de
preços no mercado educacional poderá incitar um debate mais amplo sobre o papel das escolas
particulares em um sistema educacional que deveria ser, principalmente, equitativo. Este
fenômeno pode impulsionar um movimento por maior transparência e regulação nos preços,
garantindo que as mensalidades reflitam não apenas a qualidade, mas também a responsabilidade
social e a acessibilidade. Em suma, o levantamento realizado pela Mercado Mineiro é uma janela
para um complexo cenário educacional em Belo Horizonte que merece uma reflexão aprofundada. Os
números não apenas mostram uma disparidade econômica, mas também pedem um diálogo proativo
sobre o futuro da educação na cidade e no Brasil.
"""

# Função para estruturar o texto
def estruturar_texto(texto):
    # Separar capítulos
    capitulos = re.split(r'\*\*(.*?)\*\*', texto)
    estrutura = []
    for i in range(0, len(capitulos), 2):
        pre_capitulo = capitulos[i].strip()
        if i + 1 < len(capitulos):
            titulo_capitulo = capitulos[i + 1].strip()
            if titulo_capitulo:
                estrutura.append(f"\nCapítulo: {titulo_capitulo}")
        else:
            titulo_capitulo = ""
        # Separar parágrafos dentro do capítulo
        paragrafos = re.split(r'###', pre_capitulo)
        for paragrafo in paragrafos:
            paragrafo = paragrafo.strip()
            if paragrafo:
                estrutura.append(f"\nParágrafo:\n{paragrafo}")
    return '\n'.join(estrutura)

# Aplicar a função ao texto de entrada
texto_estruturado = estruturar_texto(texto)

def salvar_como_pdf(texto, nome_arquivo):
    # Configurar o documento PDF
    pdf = SimpleDocTemplate(nome_arquivo, pagesize=A4)
    estilos = getSampleStyleSheet()
    elementos = []

    # Adicionar cada linha do texto como um parágrafo
    for linha in texto.split('\n'):
        if linha.strip():  # Ignorar linhas vazias
            paragrafo = Paragraph(linha, estilos['BodyText'])
            elementos.append(paragrafo)

    # Construir o PDF
    pdf.build(elementos)

# Exemplo de uso
salvar_como_pdf(texto_estruturado, "texto_estruturado.pdf")
